In [ ]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

def extract_color_features(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mean_color = np.mean(hsv_image, axis=(0, 1))
    return mean_color

def classify_jersey(image, kmeans_model):
    feature = extract_color_features(image)
    cluster_label = kmeans_model.predict([feature])[0]
    if cluster_label == 0:
        return 'Team 1'
    else:
        return 'Team 2'

images_path = 'sample_images/city_united'

features = []
filenames = []

for filename in os.listdir(images_path):
    image_path = os.path.join(images_path, filename)
    image = cv2.imread(image_path)
    if image is not None:
        features.append(extract_color_features(image))
        filenames.append(filename)

features = np.array(features)

n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(features)

labels = kmeans.labels_

for i, filename in enumerate(filenames):
    team = 'Team 1' if labels[i] == 0 else 'Team 2'
    print(f'{filename}: {team}')

    image_path = os.path.join(images_path, filename)
    image = cv2.imread(image_path)

    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(f'{filename}: {team}')
    plt.axis('off')
    plt.show()

pca = PCA(n_components=2)
reduced_features = pca.fit_transform(features)

plt.scatter(reduced_features[:, 0], reduced_features[:, 1], c=labels, cmap='viridis')
plt.title('K-Means Clustering of Jerseys')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.show()